# Vanilla CNN

Code for Vanilla CNN based baseline model

## Imports

In [ ]:
import numpy as np
import pandas as pd
import os, random
import cv2
import matplotlib.pyplot as plt
from tqdm import tqdm

In [ ]:
import torch
import torchvision
import torchvision.transforms as transforms

## Dataloader

In [ ]:
path = "/home/ssaoji/228_Project/Aug_Seg/Augmented_Segmented_data/"#Final_Augmented_Data/" # path to augmented data
folders = ['Scale']#['Flip','Rotate','Scale','Translation']
for f in folders:
  print(len(os.listdir(path+f)))

In [ ]:
class DataLoader():
    
    def __init__(self) -> None:
      self.path = "/home/ssaoji/228_Project/Aug_Seg/Augmented_Segmented_data/" # path to augmented data#"/content/drive/MyDrive/Course_files/228/dataset_anemia/Final_Augmented_Data/" # path to augmented data
      self.folders = ['Flip','Rotate','Scale','Translation']
      self.y_india = pd.read_excel(r'/home/ssaoji/228_Project/dataset_anemia/India/India.xlsx')#np.load("/content/drive/MyDrive/Course_files/228/dataset_anemia/y_india.npy")
      self.y_italy = pd.read_excel(r'/home/ssaoji/228_Project/dataset_anemia/Italy/Italy_copy.xlsx')#np.load("/content/drive/MyDrive/Course_files/228/dataset_anemia/y_base_italy.npy")
      self.skip = 95 #Skip value in Italy (No value)
      self.split = [0.8,0.1,0.1] #train,val,test
        
    def sample(self,num,j):
        X = np.zeros((num,640,480,3))
        Y = np.zeros(num)
        seen =[]
        for i in range(num):
          f = np.random.randint(0,len(folders))
          path = self.path+self.folders[f]
          l = os.listdir(path)
          if j ==0:
            im_p = os.listdir(path)[0:int(len(l)*self.split[j])]
          else:
            im_p = os.listdir(path)[int(len(l)*self.split[j-1]):int(len(l)*self.split[j])]
          image = random.choice(os.listdir(path))
          name_list = image.replace(".jpg","").split('_')
          country = name_list[0]
          id = int(name_list[1])
          if country =='italy':
            try:
              s = d.y_italy["Number"]==id
              s[s==True].index[-1]
              Y[i] = float(self.y_italy['Hgb'][s])/20
            except:
              continue
          elif country =='india':
            s = d.y_india["Number"]==float(id)
            s[s==True].index[-1]
            Y[i] = float(self.y_india['Hgb'][s])/20
            img = cv2.imread(path+'/' + image)
            X[i,:,:,:] =  img/255
        X = np.array(X)
        Y = np.array(Y)
        return X,Y
    
    # Generate validation samples
    def sample_v(self,num,j):
        X = np.zeros((num,640,480,3))
        Y = np.zeros(num)
        seen =[]
        for i in range(num):
            path = self.path+"Scale"
            l = os.listdir(path)
            im_p = os.listdir(path)[int(len(l)*self.split[j-1]):int(len(l)*self.split[j])]
            image = random.choice(os.listdir(path))
            name_list = image.replace(".jpg","").split('_')
            country = name_list[0]
            id = int(name_list[1])
            if country =='italy':
                try:
                  s = d.y_italy["Number"]==id
                  s[s==True].index[-1]
                  Y[i] = float(self.y_italy['Hgb'][s])/20
                except:
                  continue
            elif country =='india':
                s = d.y_india["Number"]==float(id)
                s[s==True].index[-1]
            Y[i] = float(self.y_india['Hgb'][s])/20
            temp =cv2.imread(path+'/' + image)
        X[i,:,:,:] =  temp/255 #cv2.cvtColor(temp, cv2.COLOR_BGR2HSV)/255

        X = np.array(X)
        Y = np.array(Y)
        return X,Y

In [ ]:
# Test loader
d = DataLoader()

## CNN model

In [ ]:
import torch.nn as nn
import torch.nn.functional as F


class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 16, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(16, 16, 5)
        self.fc1 = nn.Linear(293904, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 1)
        self.relu = nn.LeakyReLU()
        self.sigmoid = nn.Sigmoid()
        self.dropout = nn.Dropout(0.25)


    def forward(self, x):
        x = self.pool(self.relu(self.conv1(x)))
        x = self.pool(self.relu(self.conv2(x)))
        x = torch.flatten(x, 1) # flatten all dimensions except batch
        x = self.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.relu(self.fc2(x))
        x = self.dropout(x)
        x = self.sigmoid(self.fc3(x))
        # x = self.fc3(x)
        return x

net = Net()
net = net.cuda()

## Model Training

In [ ]:
import torch.optim as optim
# check GPU availability
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

In [ ]:
criterion = nn.MSELoss()
criterion = criterion.cuda()
# optimizer = optim.SGD(net.parameters(), lr=0.01, momentum=0.9)
optimizer = optim.Adam(net.parameters(), lr=0.001)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=2, gamma=0.9)

In [ ]:
batch_size = 100
epochs = 20
batches = 10
wandb.config = {
  "learning_rate": 0.001,
  "epochs": epochs,
  "batch_size": batch_size,
  "batches" : batches
}


d = DataLoader()
x,Y = d.sample_v(10,1)
X =np.zeros((10,3,640,480))
X[:,0,:,:] = x[:,:,:,0]
X[:,1,:,:] = x[:,:,:,1]
X[:,2,:,:] = x[:,:,:,2]
y_val = torch.tensor(Y).float().unsqueeze(1).to(device)
X_val = torch.tensor(X).float().to(device)

for epoch in tqdm(range(epochs)):  # loop over the dataset multiple times
    running_loss = 0.0
    validation_loss = 0.0
    # iters = (len(X_train)//batch_size)+1
    for i in range(batches):
        x,Y = d.sample(batch_size,0)
        X =np.zeros((batch_size,3,640,480))
        X[:,0,:,:] = x[:,:,:,0]
        X[:,1,:,:] = x[:,:,:,1]
        X[:,2,:,:] = x[:,:,:,2]
        X=X[:,:,:]
        inputs = torch.tensor(X).float().to(device)
        labels = torch.tensor(Y).float().unsqueeze(1).to(device)
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        
    with torch.no_grad():
        y_pred_val = net(X_val)
        print(y_pred_val)
        val_loss = criterion(y_pred_val, y_val)
        #       val_loss20 = criterion(y_pred_val*20, y_val*20)
        validation_loss += val_loss.item()    
        wandb.log({"loss": running_loss/batches,"loss_scaled": 400*(running_loss/batches),"val_loss":val_loss,"val_loss_scaled":val_loss*400})
        wandb.watch(net)

#       print(f'[{epoch + 1}] loss: {running_loss :.9f}, val_loss: {val_loss.item():.3f}, val_loss20: {val_loss20.item():.3f}')
        # running_loss = 0.0
        # validation_loss = 0.0
        
    scheduler.step()

In [ ]:
with torch.no_grad():
    print(net(X_val)*20)
print(y_val*20)